In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
from torchvision import transforms

class MultiInputDataset(Dataset):
    def __init__(self, csv_file, transform_rgb=None, transform_binary=None):
        self.data = pd.read_csv(csv_file)

        # Tworzenie mapowania nazw klas na liczby całkowite
        self.class_to_idx = {class_name: idx for idx, class_name in enumerate(self.data['class'].unique())}

        self.transform_rgb = transform_rgb
        self.transform_binary = transform_binary

    def __len__(self):
        return len(self.data) // 3  # Każde ziarno ma 3 obrazy

    def __getitem__(self, idx):
        # Pobierz trzy obrazy
        base_idx = idx * 3
        t_path = self.data.iloc[base_idx]['path']
        b_path = self.data.iloc[base_idx + 1]['path']
        s_path = self.data.iloc[base_idx + 2]['path']

        t_image = Image.open(t_path).convert("RGB")
        b_image = Image.open(b_path).convert("RGB")
        s_image = Image.open(s_path).convert("L")  # Obraz binarny

        # Transformacje
        if self.transform_rgb:
            t_image = self.transform_rgb(t_image)
            b_image = self.transform_rgb(b_image)
        if self.transform_binary:
            s_image = self.transform_binary(s_image)

        # Pobierz nazwę klasy i przekształć na indeks numeryczny
        class_name = self.data.iloc[base_idx]['class']
        label = self.class_to_idx[class_name]  # Mapowanie nazwy klasy na numer
        label = torch.tensor(label, dtype=torch.long)  # Konwersja na tensor PyTorch

        return t_image, b_image, s_image, label

#Krok 2: Transformacje dla obrazów RGB i binarnych:
# Transformacje dla obrazów RGB
transform_rgb = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Transformacje dla obrazów binarnych
transform_binary = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [2]:
import torch
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

def test_model(path_model, model_class, test_loader, class_to_idx, device="cpu"):
    """
    Testuje model na zbiorze testowym i wyznacza macierz pomyłek.

    Args:
        path_model (str): Ścieżka do pliku .pth z zapisanymi wagami modelu.
        model_class (class): Klasa modelu użytego do trenowania.
        test_loader (DataLoader): DataLoader dla zbioru testowego.
        class_to_idx (dict): Mapowanie nazw klas na liczby całkowite.
        device (str): Urządzenie ("cuda" lub "cpu").

    Returns:
        cm: Macierz pomyłek.
        y_true: Rzeczywiste etykiety.
        y_pred: Przewidywane etykiety.
    """
    # Inicjalizacja modelu
    model = model_class().to(device)

    # Załaduj wagi modelu
    model.load_state_dict(torch.load(path_model, map_location=device))
    model.eval()  # Ustawienie modelu w tryb ewaluacji

    y_true = []
    y_pred = []

    with torch.no_grad():
        for t_image, b_image, s_image, labels in test_loader:
            t_image, b_image, s_image, labels = (
                t_image.to(device),
                b_image.to(device),
                s_image.to(device),
                labels.to(device)
            )

            # Oblicz predykcje
            outputs = model(t_image, b_image, s_image)
            _, predicted = torch.max(outputs, 1)

            # Zbierz rzeczywiste i przewidywane etykiety
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    # Wyznaczenie macierzy pomyłek
    cm = confusion_matrix(y_true, y_pred)

    return cm, y_true, y_pred


# Załaduj dane
test_dataset = MultiInputDataset("CSV/dataset/test.csv", transform_rgb=transform_rgb, transform_binary=transform_binary)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# Wywołanie funkcji testującej
cm, y_true, y_pred = test_model(
    path_model="training_results/best_model_efficientnet_b0.pth",
    model_class=lambda: MultiInputModel(num_classes=len(test_dataset.class_to_idx), base_model="efficientnet_b0"),
    test_loader=test_loader,
    class_to_idx=test_dataset.class_to_idx,
    device="cuda"
)

# Wyświetlenie macierzy pomyłek
class_names = list(test_dataset.class_to_idx.keys())
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Przewidywane klasy")
plt.ylabel("Rzeczywiste klasy")
plt.title("Macierz pomyłek")
plt.show()

# Inne metryki
print("Dokładność:", accuracy_score(y_true, y_pred))
print("\nRaport klasyfikacji:")
print(classification_report(y_true, y_pred, target_names=class_names))


NameError: name 'MultiInputModel' is not defined

In [3]:
model_data = torch.load("training_results/best_model_efficientnet_b0.pth")
print(type(model_data))

<class 'collections.OrderedDict'>


/tmp/ipykernel_163674/1989228468.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_data = torch.load("training_results/best_model_efficientnet_b0.pth")
